In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import os
from PIL import Image
from sklearn.metrics import roc_auc_score, classification_report, accuracy_score, f1_score, precision_score, recall_score
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from transformers import ViTForImageClassification, ViTFeatureExtractor
import matplotlib.pyplot as plt

In [ ]:
from huggingface_hub import login

login("")


In [ ]:
import timm
import torch
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
from timm.layers import SwiGLUPacked
from PIL import Image

# need to specify MLP layer and activation function for proper init
model = timm.create_model("hf-hub:paige-ai/Virchow2", pretrained=True, mlp_layer=SwiGLUPacked, act_layer=torch.nn.SiLU)
model = model.eval()

transforms = create_transform(**resolve_data_config(model.pretrained_cfg, model=model))

image = Image.open("/home/iambrink/NOH_Thyroid_Cancer_Data/TAN/001/IMG_20220623_134910.jpg")
image = transforms(image).unsqueeze(0)  # size: 1 x 3 x 224 x 224

output = model(image)  # size: 1 x 261 x 1280

class_token = output[:, 0]    # size: 1 x 1280
patch_tokens = output[:, 5:]  # size: 1 x 256 x 1280, tokens 1-4 are register tokens so we ignore those

# concatenate class token and average pool of patch tokens
embedding = torch.cat([class_token, patch_tokens.mean(1)], dim=-1)  # size: 1 x 2560


In [ ]:
model = model.to("cuda")
image = image.to("cuda")

with torch.inference_mode(), torch.autocast(device_type="cuda", dtype=torch.float16):
  output = model(image)

class_token = output[:, 0]
patch_tokens = output[:, 5:]

embedding = torch.cat([class_token, patch_tokens.mean(1)], dim=-1)

# the model output will be fp32 because the final operation is a LayerNorm that is ran in mixed precision
# optionally, you can convert the embedding to fp16 for efficiency in downstream use
embedding = embedding.to(torch.float16)


In [ ]:
model = timm.create_model("hf-hub:paige-ai/Virchow2")


# training

semi faster one (32 batch)

In [ ]:
import timm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
from timm.layers import SwiGLUPacked
import pandas as pd
import os
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Enable benchmark mode in cuDNN
torch.backends.cudnn.benchmark = True

# Set paths
data_path = "/home/iambrink/NOH_Thyroid_Cancer_Data/CSV-files/Thyroid_Cancer_TAN&NOH_file.csv"
base_image_path = "/home/iambrink/NOH_Thyroid_Cancer_Data/superdata/"

# Load and clean the dataset
df = pd.read_csv(data_path)
df = df.dropna(subset=["Surgery diagnosis in number"])

# Split into train and validation sets (80% train, 20% validation)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["Surgery diagnosis in number"])

# Custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, dataframe, base_path, transform=None):
        self.dataframe = dataframe
        self.base_path = base_path
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = os.path.join(self.base_path, self.dataframe.iloc[idx]["image_path"].replace("\\", "/"))
        image = Image.open(img_path).convert("RGB")
        label = int(self.dataframe.iloc[idx]["Surgery diagnosis in number"])
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.long)

# Load the pre-trained model
model = timm.create_model(
    "hf-hub:paige-ai/Virchow2", 
    pretrained=True, 
    mlp_layer=SwiGLUPacked, 
    act_layer=torch.nn.SiLU
)

# Modify the model for binary classification
num_features = model.num_features
model.head = nn.Linear(num_features, 2)

# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define transformations using timm's data config
config = resolve_data_config(model.pretrained_cfg, model=model)
transform = create_transform(**config)

# Create datasets
train_dataset = CustomDataset(train_df, base_image_path, transform=transform)
val_dataset = CustomDataset(val_df, base_image_path, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=16, pin_memory=True, persistent_workers=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=16, pin_memory=True, persistent_workers=True)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-5, weight_decay=1e-5)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=4, verbose=True)

# Automatic mixed precision
scaler = torch.cuda.amp.GradScaler()

# Training setup
num_epochs = 25
best_val_acc = 0.0  # Store the best validation accuracy

for epoch in range(num_epochs):
    model.train()
    train_loss, correct, total = 0, 0, 0

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            outputs = model(images)  # Expected shape: [batch_size, 261, 2]
            outputs = outputs[:, 0, :]  # Use the class token [batch_size, 2]
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)
        progress_bar.set_postfix(loss=loss.item(), acc=100. * correct / total)

    train_acc = 100. * correct / total
    print(f"Epoch {epoch+1}: Train Loss={train_loss/len(train_loader):.4f}, Train Accuracy={train_acc:.2f}%")

    # Validation phase
    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)[:, 0, :]
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_correct += predicted.eq(labels).sum().item()
            val_total += labels.size(0)

    val_acc = 100. * val_correct / val_total
    print(f"Epoch {epoch+1}: Validation Loss={val_loss/len(val_loader):.4f}, Validation Accuracy={val_acc:.2f}%")

    # Step scheduler based on validation accuracy
    scheduler.step(val_acc)

    # Save the best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "virchow2_best_test.pth")
        print(f"New best model saved with accuracy: {best_val_acc:.2f}%")

print("Training Complete. Best model saved as 'virchow2_best_test.pth'.")


/home/iambrink/miniconda3/envs/tf-gpu/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: [Errno 2] No such file or directory: '/home/iambrink/NOH_Thyroid_Cancer_Data/Thyroid_Cancer_TAN&NOH_file.csv'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import os
from PIL import Image
from sklearn.metrics import roc_auc_score, accuracy_score
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import timm
import matplotlib.pyplot as plt

# Set paths
data_path = '/home/iambrink/NOH_Thyroid_Cancer_Data/Thyroid_Cancer_TAN&NOH_file.csv'
base_image_path = '/home/iambrink/NOH_Thyroid_Cancer_Data/'

# Load dataset
df = pd.read_csv(data_path)
df = df.dropna(subset=['Surgery diagnosis in number'])  # Drop NaN labels

# Split data into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  
])

# Custom Dataset Class
class CustomDataset(Dataset):
    def __init__(self, dataframe, base_path, transform=None):
        self.dataframe = dataframe
        self.base_path = base_path
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = os.path.join(self.base_path, self.dataframe.iloc[idx]['image_path'].replace('\\', '/'))
        image = Image.open(img_path).convert('RGB')
        label = int(self.dataframe.iloc[idx]['Surgery diagnosis in number'])  

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long)

# Create datasets and loaders
train_dataset = CustomDataset(train_df, base_image_path, transform=transform)
test_dataset = CustomDataset(test_df, base_image_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load Virchow2 Model without pretrained weights
device = "cuda" if torch.cuda.is_available() else "cpu"

model = timm.create_model("hf_hub:paige-ai/Virchow2", pretrained=False)
model = model.to(device)

# Modify final layer to match classification task (Binary = 2, Multi-class = 3416)
num_classes = 2  # Change this if you have more than two classes
model.blocks[-1].mlp.fc2 = torch.nn.Linear(1280, num_classes).to(device)
model.head = nn.Linear(model.num_features, 2)
# Define Optimizer & Loss Function
optimizer = optim.AdamW(model.parameters(), lr=5e-5, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

# Tracking metrics
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []
val_aucs = []

# Training loop
num_epochs = 50
best_val_auc = 0.0

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct_predictions = 0

    with tqdm(total=len(train_loader), desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch') as pbar:
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            predictions = torch.argmax(outputs, dim=1)
            correct_predictions += (predictions == labels).sum().item()

            pbar.set_postfix(loss=loss.item())
            pbar.update(1)

    # Calculate training metrics
    train_loss /= len(train_loader)
    train_acc = correct_predictions / len(train_dataset)
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct_predictions = 0
    val_labels = []
    val_preds = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            predictions = torch.softmax(outputs, dim=1)[:, 1]  # Get probability for class 1
            val_correct_predictions += (torch.argmax(outputs, dim=1) == labels).sum().item()

            val_labels.extend(labels.cpu().numpy())
            val_preds.extend(predictions.cpu().numpy())

    # Compute validation metrics
    val_loss /= len(test_loader)
    val_acc = val_correct_predictions / len(test_dataset)
    val_auc = roc_auc_score(val_labels, val_preds)

    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    val_aucs.append(val_auc)

    print(f'Epoch [{epoch + 1}/{num_epochs}], '
          f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val AUC: {val_auc:.4f}')

    # Save best model based on AUC score
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        torch.save(model.state_dict(), 'best_Virchow2_Thyroid.pth')
        print(f"Model saved with best validation AUC: {best_val_auc:.4f}")

# Plot Loss and Accuracy over Epochs
epochs = range(1, num_epochs + 1)

plt.figure(figsize=(12, 5))

# Plot Loss
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label='Training Loss')
plt.plot(epochs, val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()

# Plot Accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, label='Training Accuracy')
plt.plot(epochs, val_accuracies, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy over Epochs')
plt.legend()

plt.tight_layout()
plt.show()
